In [1]:
from PyMLB import *

Following ID's will be used for sample work: 
- teamId : 141 (for Toronto Blue Jays)
- gameId: 634642
- personId: 547943 
- sportId: 1
- leagueId: 103 (AL) or 104 (NL) 

### Attendance Class

In [61]:
class attendance():
    def __init__(self,
                 id,
                 teamOrleague = "team",
                 date = None,
                 startDate = None,
                 endDate = None,
                 season = None,
                 field = "Ytd"):
        self.Id = id
        self.teamOrleague = teamOrleague
        self.date = date
        self.startDate = startDate
        self.endDate = endDate
        self.season = season
        self.field = field
        
        if (startDate is not None and endDate is None) or (startDate is None and endDate is not None):
            ValueError("Specify both Start Date and End Date in MM/DD/YYYY format.")
        
        if not any([self.date, self.startDate, self.endDate, self.season]):
            self.date = today
        
    def get_attendance(self):
        url = alt_base_url + "v1/attendance"
        attedance_field = "attendanceAverage" + self.field
        suffix = ""
        
        attendance_dict = {}     

        if self.date is not None:
            suffix += "?date={}".format(self.date)

        if self.startDate is not None and self.endDate is not None:
            suffix += "?startDate={startDate}&endDate={endDate}".format(startDate = self.startDate,
                                                                        endDate = self.endDate)    

        if self.season is not None:
            if type(self.season) is list:
                suffix += "?season={}".format(",".join(str(s) for s in self.season))
            else:
                suffix += "?season={}".format(self.season)
                
        if self.teamOrleague == "team":
            if type(self.Id) is list:
                ID_str = ",".join(str(i) for i in self.Id)
            else:
                ID_str = self.Id

            suffix += "&teamId={teamId}".format(teamId = ID_str)           

            attendance_content = get_json(url + suffix)
            records = attendance_content["records"]

            for record in records:
                team = record["team"]["id"]
                attendance = record[attedance_field]

                if self.date is not None:
                    attendance_dict.update({team: {self.date: attendance}})

                if self.startDate is not None and self.endDate is not None:
                    attendance_dict.update({team: {"{}".format(self.startDate + " to " + self.endDate):attendance}})        

                if self.season is not None:
                    if type(self.season) is list:
                        season_dict = {}
                        for s in self.season:
                            season_dict.update({s: attendance})
                        attendance_dict.update({team: season_dict})
                    else:
                        attendance_dict.update({team: {self.season: attendance}})

        if self.teamOrleague == "league":
            if type(self.Id) is not list:
                Ids = [self.Id]
            else: 
                Ids = self.Id

            for i in Ids:
                suffix += "&leagueId={leagueId}".format(leagueId = i)
                attendance_content = get_json(url + suffix)
                records = attendance_content["records"]

                for record in records:
                    league = i
                    attendance = record[attedance_field]

                    if self.date is not None:
                        attendance_dict.update({league: {self.date: attendance}})

                    if self.startDate is not None and self.endDate is not None:
                        attendance_dict.update({league: {"{}".format(self.startDate + " to " + self.endDate):attendance}})    

                    if self.season is not None:
                        if type(self.season) is list:
                            season_dict = {}
                            for s in self.season:
                                season_dict.update({s: attendance})
                            attendance_dict.update({league: season_dict})
                        else:
                            attendance_dict.update({league: {self.season: attendance}})

        #self.attendance = attendance_dict
        return(attendance_dict)
    
    def print(self):
        attendance = self.get_attendance()
        
        for tOrl, att in attendance.items():
            if self.teamOrleague == "team":
                print("For team {}:".format(tOrl))
                
                for d, a in att.items():
                    print("\tAttendance was {a} in/on {d}".format(a = a, d = d))


### Linescore Class:

In [99]:
class linescore():
    def __init__(self, gamePk):
        self.gamePk = gamePk
        
    def get_linescore(self):
        team_url = base_url + "v1/schedule?gamePk={gamePk}".format(gamePk = self.gamePk)
        team_content = get_json(team_url)

        teams = team_content["dates"][0]["games"][0]["teams"]
        team_dict = {"teams": {k: {x: y for x,y in v["team"].items() if x == "id" or x == "name"} 
                               for k, v in teams.items()}}

        ls_url = base_url + "/v1/game/{gamePk}/linescore".format(gamePk = self.gamePk)
        linescore_content = get_json(ls_url)

        linescore_dict = {}

        innings = linescore_content["innings"]

        for inning in innings:
            inn_count = inning["ordinalNum"]
            home = {x: y for x, y in inning["home"].items() if x != "leftOnBase"} 
            away = {x: y for x, y in inning["away"].items() if x != "leftOnBase"} 

            linescore_dict.update({inn_count: {"home": home, "away": away}})

        linescore_dict.update(team_dict)

        return(linescore_dict)
    
    def print(self):
        ls = self.get_linescore()
        
        teams_dict = {k: v['name'] for k, v in ls['teams'].items()}
        ls = {k: v for k, v in ls.items() if k != "teams"}
        
        home_total_run = 0
        home_total_hits = 0
        home_total_errors = 0
        
        away_total_run = 0
        away_total_hits = 0
        away_total_errors = 0
        
        header = ""
        home = "home"
        away = "away"
        
        for k, v in ls.items():
            header += "\t" + k
            
            for team, stat in v.items():
                if team == "home":
                    home += "\t" + str(v[team]['runs'])
                    home_total_run += v[team]['runs']
                    home_total_hits += v[team]['hits']
                    home_total_errors += v[team]['errors']
                    
                if team == "away":
                    away += "\t" + str(v[team]['runs'])
                    away_total_run += v[team]['runs']
                    away_total_hits += v[team]['hits']
                    away_total_errors += v[team]['errors']
                    
        header += "\tR\tH\tE"
        home += "\t" + "\t".join([str(home_total_run),
                                    str(home_total_hits),
                                    str(home_total_errors)])
        away += "\t" + "\t".join([str(away_total_run),
                                    str(away_total_hits),
                                    str(away_total_errors)])

        
        print(header)
        print(away)
        print(home)
        print("\n")
        
        for team, name in teams_dict.items():
            print("{team}: {name}".format(team = team, name = name))

### Player Stat

In [98]:
class player_stat():
    def __init__(self, personId, gamePk):
        self.personId = personId
        self.gamePk = gamePk
        
    def lookup(self):
        game_stat_url = base_url + "v1/people/{playerId}/stats/game/{gamePk}".format(playerId = self.personId, gamePk = self.gamePk)
        game_stat_content = get_json(game_stat_url)
        game_stat = game_stat_content["stats"]
        
        stats = [s for s in game_stat if s["splits"]]
    
        stat_dict = {}
        
        try:    
            for s in stats:        
                t = s["group"]["displayName"]
                splits = s["splits"]
                stat = splits[0]["stat"]
                stat_dict.update({t:stat})
            return(stat_dict)
        
        except:
            print("The player did not play in the stated game.")

        #stat = [d for d in splits if "stat" in d.keys()][0]
        #stat_dict.update({t:stat["stat"]})    